## Approach 1: Dynamic Programming

Throughout this document, the following packages are required:

In [1]:
import numpy as np
import scipy, math
from scipy.stats import poisson
from scipy.optimize import minimize

### Heterogeneous Exponential Case

The following functions implement the heterogeneous exponential case (Theorem 2.28).

Laat $B_j \sim \text{Exp}(\mu_j)$ de behandeltijd van de $j$-de klant. Er geldt:
\begin{align*}
p_{k\ell,i}(t)
= \mathbb{P}_{i}(N_t = \ell\mid N_0 = k)
&= \mathbb{P}\left(\sum_{j=i-k+1}^{i-\ell+2}B_j \leq t\right) - \mathbb{P}\left(\sum_{j=i-k+1}^{i-\ell+3}B_j \leq t\right) \\
&= \sum_{j=i-k+1}^{i-\ell+2}\frac{c_{i-k+1,k-\ell+1,j}}{\mu_j}(1 - e^{-\mu_j t}) - \sum_{j=i-k+1}^{i-\ell+3}\frac{c_{i-k+1,k-\ell+2,j}}{\mu_j}(1 - e^{-\mu_j t}),
\end{align*}
de $p_{k1,i}(t)$ klopt wel.

In [2]:
def trans_prob_het(t,i,k,mu):
    """Computes the transition probabilities (Prop. 2.25)."""
    
    p = [phi(i-k+1,k-l+1,t,mu) / mu[i-l+1] for l in range(2,k+2)]
    
    return [1 - np.sum(p)] + p


In [3]:
t = 0.3
i = 4
k = 3
mu = np.arange(0.8,1.2,5)

In [4]:
trans_prob_het(t,i,k,mu)

NameError: name 'phi' is not defined

In [5]:
# helper functions
def phi(k,l,s,mu):
    return np.sum([c(k,l,j,mu) * np.exp(-mu[j-1] * s) for j in range(k,k+l+1)])

def psi(j,t,mu):
    return (1 - np.exp(-mu[j-1] * t)) / mu[j-1]

def c(k,l,j,mu):
    """Computes the weights c of phi recursively (Lemma 2.23)."""

    # storage indices
    k_, l_, j_ = k - 1, l, j - 1
    
    if c_stored[k_][l_][j_] != None:
        pass
    elif k == j and not l:
        c_stored[k_][l_][j_] = mu[k_]
    elif l:
        if j >= k and j < k + l:
            c_stored[k_][l_][j_] = c(k,l-1,j,mu) * mu[k_+l_] / (mu[k_+l_] - mu[j-1])
        elif k + l == j:
            c_stored[k_][l_][j_] = np.sum([c(k,l-1,m,mu) * mu[j-1] / (mu[m-1] - mu[j-1]) for m in range(k,k+l)])
    
    return c_stored[k_][l_][j_]

def trans_prob_het(t,i,k,mu):
    """Computes the transition probabilities (Prop. 2.25)."""
    
    p = [phi(i-k+1,k-l+1,t,mu) / mu[i-l+1] for l in range(2,k+2)]
    
    return [1 - np.sum(p)] + p

def cost_het(t,i,k,mu,omega,n):
    """Computes the cost when t is the next interarrival time."""
    
    f = t - np.sum([c(i-k+1,k-1,j,mu) * psi(j,t,mu) / mu[j-1] for j in range(i-k+1,i+1)])
    #g = 0 ## alternative
    #for l in range(k-1):
    #   g += (k - l - 1) * np.sum([c(i-k+1,l,j,mu) * psi(j,t,mu) / mu[i-k+l] for j in range(i-k+1,i-k+l+2)])
    h = np.sum(1 / mu[i-k:i-1])
    
    p = trans_prob_het(t,i,k,mu)
    cost = omega * f[0] + (1 - omega) * h + np.sum([Cstar_het(i+1,l,mu,omega,n) * p[l-1] for l in range(1,k+2)])
    
    return cost
    
def Cstar_het(i,k,mu,omega,n):
    """Implements the Heterogeneous Exponential Case."""
    
    mu = np.array(mu)

    if C_matrix[i-1][k-1] != None: # retrieve stored value
        pass
    elif i == n: # initial condition
        C_matrix[i-1][k-1] = (1 - omega) * np.sum(1 / mu[i-k:i-1])
        # C_matrix[i-1][k-1] = (1 - omega) * np.sum([(k - l - 1) / mu[n-k+l] for l in range(k)]) ## alternative
    else:
        optimization = minimize(cost_het,0,args=(i,k,mu,omega,n),bounds=((0,500),))
        C_matrix[i-1][k-1] = optimization.fun
        minima[i-1][k-1] = optimization.x[0]
        print(i,k,minima[i-1][k-1],C_matrix[i-1][k-1]) # displays C_i(k) and interarrival time
    
    return C_matrix[i-1][k-1]


In [6]:
def trans_prob_het(t,i,k,mu):
    """Computes the transition probabilities (Prop. 2.25)."""
    
    p = [phi(i-k+1,k-l+1,t,mu) / mu[i-l+1] for l in range(2,k+2)]
    
    return [1 - np.sum(p)] + p

def trans_prob_het2(t,i,k,mu):
    
    p = [0] * (k+1)
    
#     p[0] = sum([sum([c(i-k+1,m,j,mu) * np.exp(-mu[j-1] * t) / mu[j-1+m] for j in range(i-k+1,i-k+1+m+1)]) for m in range(k)])
#     p[0] = 1 - sum([sum([c(i-k+1,m,j,mu) * np.exp(-mu[j-1] * t) / mu[j+m-1] for j in range(i-k+1,i-k+m+2)]) for m in range(k)])
    
#     p[0] = np.sum([c(i-k+1,k-1,j,mu) * psi(j,t,mu) for j in range(i-k+1,i+1)])
    p[0] = 1 - sum([sum([c(i-k+1,m,j,mu) * np.exp(-mu[j-1] * t) / mu[i-k+m] for j in range(i-k+1,i-k+m+2)]) for m in range(k)])
    
    
    for l in range(2,k+1):
        p[l-1] = np.sum([c(i-k+1,k-l,j,mu) * psi(j,t,mu) for j in range(i-k+1,i-l+2)]) \
                    - np.sum([c(i-k+1,k-l+1,j,mu) * psi(j,t,mu) for j in range(i-k+1,i-l+3)])
    
    p[k] = np.exp(-mu[i-k] * t)
    
    return p

In [10]:
t = 3.82
i = 4
k = 4
mu = np.linspace(0.5,1.5,n)

print(trans_prob_het(t,i,k,mu))
print(trans_prob_het2(t,i,k,mu))

[0.36369407962193834, 0.1379052471787745, 0.16812577567696796, 0.18219451092685682, 0.14808038659546247]
[0.36369407962193834, 0.13790524717877783, 0.16812577567696874, 0.18219451092685723, 0.14808038659546247]


In [48]:
g = 0
for l in range(k-1):
    g += (k - l - 1) * np.sum([c(i-k+1,l,j,mu) * psi(j,t,mu) / mu[i-k+l] for j in range(i-k+1,i-k+l+2)])
print(g)

g2 = (k-1) * psi(i-k+1,t,mu)
for l in range(1,k-1):
    g2 += (k - l - 1) * (np.sum([c(i-k+1,l,j,mu) * (psi(j,t,mu)) / mu[j-1] for j in range(i-k+1,i-k+l+2)]) \
                         - np.sum([c(i-k+1,l-1,j,mu) * (psi(j,t,mu)) / mu[j-1] for j in range(i-k+1,i-k+l+1)]))

print(g2)

7.399050613835084
7.399050613835085


With this code, we can compute the optimal cost $C_{1}(1)$ for the heterogeneous case dynamically. An example:

In [9]:
omega = 0.7
n = 5
mu = np.linspace(0.5,1.5,n)
# mu = np.array([1e-1 * i for i in range(n)])
# mu = mu - np.mean(mu) + 1
print("omega =", omega, "and mu =", mu, "\n")

print("(i,k,t*,C)")
C_matrix = [[None for k in range(n)] for i in range(n)]
minima = [[None for k in range(n)] for i in range(n)]
c_stored = [[[None for j in range(n)] for l in range(n)] for k in range(n)]

# compute values
for i in range(1,n+1):
    for k in range(1,i+1):
        Cstar_het(i,k,mu,omega=omega,n=n)

# cost
print("\nCost:", C_matrix[0][0])

omega = 0.7 and mu = [0.5  0.75 1.   1.25 1.5 ] 

(i,k,t*,C)
4 1 0.28533998550485146 0.19973796860569054
4 2 0.983740141087462 0.6625280023536871
3 1 0.5074960827719017 0.5549860524936788
4 3 1.9759634367783472 1.2456395920571923
3 2 1.567667288718443 1.239582872454175
2 1 0.7335205781113533 1.0684505384301732
4 4 3.4509182821195994 2.112890155013404
3 3 3.177705697955406 2.214345675064946
2 2 2.3486391429776945 2.143643832599409
1 1 1.1396884158870093 1.8662352954388974

Cost: 1.8662352954388974


We can also compute the minimal cost when scheduling all clients instantaneously:

In [98]:
def compute_probN_het(t,mu):
    """Computes P(N_ti = j) for i=1,...,n and j=1,...,i."""
    
    n = len(mu)
    p = np.zeros((n,n))
    p[0][0] = 1
    
    for i in range(2,n+1):
        
        x = t[i-1] - t[i-2]
        
        # j = 1
        for k in range(1,i):
            p[i-1][0] += np.sum([c(i-k,k-1,m,mu) * psi(m,x,mu) for m in range(i-k,i)]) * p[i-2][k-1]
        
        # j = 2,...,i
        for j in range(2,i+1):
            p[i-1][j-1] = np.sum([(phi(i-k,k-j+1,x,mu) / mu[i-j]) * p[i-2][k-1] for k in range(j-1,i)])
            
    return p

def static_cost_het(t,mu,omega):
    """Computes the cost of the optimal static schedule."""
    
    mu, n = np.array(mu), len(mu)
    EW, EI = np.zeros(n), np.zeros(n)    
    p = compute_probN_het(t,mu)
    
    for i in range(2,n+1):
        
        x = t[i-1] - t[i-2]
        EW[i-2] = np.sum([np.sum(1 / mu[i-j:i-1]) * p[i-1][j-1] for j in range(2,i+1)])
        
        for j in range(1,i):
            f = np.sum([c(i-j,j-1,m,mu) * (x - psi(m,x,mu)) / mu[m-1] for m in range(i-j,i)])
            EI[i-2] += f * p[i-2][j-1]
            
    return omega * np.sum(EI) + (1 - omega) * np.sum(EW)


Again we give an example, in which we compare the dynamic program with the static program:

In [105]:
np.linspace(0.5,1.5,5)

array([0.5 , 0.75, 1.  , 1.25, 1.5 ])

In [107]:
omega = 0.5
n = 15
mu = np.linspace(0.5,1.5,n)
# Delta = 1.5
# mu = np.linspace(1 - Delta/2,1 + Delta/2,n)
# mu = mu[::-1]
# mu = np.random.permutation(mu)

C_matrix = [[None for k in range(n)] for i in range(n)]
minima = [[None for k in range(n)] for i in range(n)]
c_stored = [[[None for j in range(n)] for l in range(n)] for k in range(n)]

# compute values
for i in range(1,n+1):
    for k in range(1,i+1):
        Cstar_het(i,k,mu,omega=omega,n=n)

# cost
dynamic_cost = C_matrix[0][0]

c_stored = [[[None for j in range(n)] for l in range(n)] for k in range(n)]
optimization = minimize(static_cost_het,range(n),args=(mu,omega), bounds=(((0,0),) + (((0,None)),) * (n-1)))
print(optimization)

static_cost = optimization.fun

print("\nmu:",mu)
print("omega:",omega)
print("\nDynamic Cost:", round(dynamic_cost,2))
print("Static Cost:", round(static_cost,2))

ratio = dynamic_cost / static_cost
print("ratio:", round(ratio,2))

14 1 0.48520318201686596 0.24260151319598955
14 2 1.2055856594096712 0.7462764178567411
13 1 0.6349244550238325 0.5600599912282797
14 3 1.9734022867996064 1.2443901511538513
13 2 1.4538921535631517 1.1112536959143329
12 1 0.6865249206908528 0.9033221283048873
14 4 2.7876985125578875 1.7550331626015232
13 3 2.3090984569761632 1.6534463195712137
12 2 1.5604078089903597 1.4895740641291864
11 1 0.7290849959464828 1.2678647795246674
14 5 3.653202333821924 2.287117261505871
13 4 3.2100763182846044 2.2090480448449874
12 3 2.4738489637460113 2.0678848968961114
11 2 1.6582783138652126 1.8914336107320326
10 1 0.7751998881014366 1.6554702253268312
14 6 4.573986630907299 2.847431645865811
13 5 4.165615595557914 2.7889617086852354
12 4 3.438906302606088 2.66273408500748
11 3 2.63300276953684 2.5091316723665247
10 2 1.7663166604708531 2.32113116445501
9 1 0.8273209905637346 2.069130742699926
14 7 5.566315984729083 3.442515739801192
13 6 5.188052728726724 3.401536614430916
12 5 4.466381470183038 3.28

In [ ]:
# TODO: Old code


# def C_static_het(times,i,k,mu,omega=0.5,n=15):
#     """
#     Implements the Heterogeneous Exponential Case.
#     """
#     mu = np.array(mu)

# #     print("i",i)
#     if C_matrix[i-1][k-1] != None: # retrieve stored value
#         pass
#     elif i == n: # initial condition
#         C_matrix[i-1][k-1] = (1 - omega) * np.sum([1 / mu[j-1] for j in range(i-k+1,i)])
#     else:
# #         print("n",n)
# #         print("i:",i)
#         t = times[i]
# #         print(t)
        
#         # helper function
#         psi = lambda j,t: (1 - np.exp(-mu[j-1] * t)) / mu[j-1]
        
#         # compute f and g
#         f = np.sum([c(i-k+1,k-1,j,mu) * (t - psi(j,t)) / mu[j-1] for j in range(i-k+1,i+1)])
#         g = np.sum([1 / mu[j-1] for j in range(i-k+1,i)])
        
#         p = trans_prob_het(t,i,k,mu)
#         cost = omega * f + (1 - omega) * g + C_static_het(times,i+1,1,mu,omega,n) * p[0]
    
#         for l in range(2,k+2):
# #             print(i)
#             cost += C_static_het(times,i+1,l,mu,omega,n) * p[l-1]

#         C_matrix[i-1][k-1] = cost
# #         print(i,k,minima[i-1][k-1],C_matrix[i-1][k-1]) # displays C_i(k) and interarrival time
    
# #     print(i,k,C_matrix[i-1][k-1])
#     return C_matrix[i-1][k-1]


# t = [ 0.          ,4.65288472,  8.33895249, 11.07420249, 13.03410038]
# n = len(t)
# mu =  np.linspace(0.5,1.5,n)
# omega = 0.1

# C_matrix = [[None for k in range(n)] for i in range(n)]
# c_stored = [[[None for j in range(n)] for l in range(n)] for k in range(n)]

# print("Cost:",C_static_het(t,1,1,mu,omega,n))

# # for i in range(1,n+1):
# #     for k in range(1,i+1):
# #         print(t[i-1])
# #         C_static_het(t[i-1],i,k,mu,omega=omega,n=n)

# C_matrix